In [85]:
# Basic imports
import sqlite3
from pathlib import Path
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import OllamaEmbeddings
import sys
sys.path.append(r"C:\Users\ishan\Automation\SIH25\RAG")  # So we can import rag_script

from RAG import rag_query
from RAG import vector_store
retriever = vector_store.as_retriever(search_kwargs={"k": 5})



# SQLite DB file
DB_FILE = Path("conversation_memory.db")

# Initialize SQLite connection
conn = sqlite3.connect(DB_FILE, check_same_thread=False)
cursor = conn.cursor()

# Create conversation table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS conversation_history (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT NOT NULL,
    role TEXT NOT NULL,  -- 'user' or 'assistant'
    text TEXT NOT NULL,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
""")
conn.commit()


In [86]:
# Initialize LLM (Google Gemini)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Prompt template
template = """
You are an academic assistant. Answer in the language specified.
Use the context to answer the question accurately.

Context: {context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])


In [87]:
def get_conversation_history(user_id, last_n=5):
    cursor.execute("""
        SELECT role, text FROM conversation_history
        WHERE user_id = ?
        ORDER BY timestamp DESC
        LIMIT ?
    """, (user_id, last_n))
    rows = cursor.fetchall()
    # Reverse to chronological order
    return [{"role": r[0], "text": r[1]} for r in reversed(rows)]

def save_turn(user_id, role, text):
    cursor.execute("""
        INSERT INTO conversation_history (user_id, role, text)
        VALUES (?, ?, ?)
    """, (user_id, role, text))
    conn.commit()


In [88]:
def rg_generate(user_id, rag_output, language="en"):
    # Context is already a list of strings from rag_query
    context = "\n".join(rag_output["context"])
    question = rag_output["query"]

    # Updated prompt (forces grounding to docs)
    final_prompt = f"""
You are an academic assistant. 
Answer the question as much as possible with the given context and if the context was used state where it was mentioned and if it was not used then state clearly   . 
"

Answer in {language}.

Context:
{context}

Question: {question}
Answer:
"""

    # Call LLM properly
    from langchain.schema import HumanMessage
    response = llm.invoke([HumanMessage(content=final_prompt)])
    response_text = response.content

    # Save conversation to DB
    save_turn(user_id, "user", question)
    save_turn(user_id, "assistant", response_text)

    # Extract sources from metadata
    sources = [meta.get("source", "N/A") for meta in rag_output["metadata"]]
    return response_text, sources


In [89]:
user_id = "user_123"
user_query = "tell me the syllabus of Software Engineering subject."

# 1️⃣ Retrieve RAG output
rag_output = rag_query(user_query, retriever)

print("🔎 RAG Output Debug:")
print("Query:", rag_output["query"])
print("Context count:", len(rag_output["context"]))
for i, ctx in enumerate(rag_output["context"], 1):
    print(f"\n--- Context {i} ---\n{ctx[:500]}")

# 2️⃣ Generate response
answer, sources = rg_generate(user_id, rag_output, language="en")

print("\nAnswer:\n", answer)
print("\nSources:\n", sources)


🔎 RAG Output Debug:
Query: tell me the syllabus of Software Engineering subject.
Context count: 0

Answer:
 The provided context is empty. Therefore, I cannot tell you the syllabus of the Software Engineering subject based on the given context.

Sources:
 []
